# Funding Channel Analysis

This notebook documents the weekly-frequency exercise linking TIPS-Treasury mispricing persistence to synthetic funding proxies.

## Data Overview
- `_data/mispricing_basis.csv` resampled to weekly averages.
- `_output/mispricing_report.html` for benchmark persistence metrics and event windows.
- `outputs/strategy3/concentration_metrics.csv` for tenor-level liquidity measures.
- Synthetic funding proxies located in `inputs/funding_channels/`.
- WRDS probe failed; see `_output/funding_channel_failure_log.json`.
- Outputs saved to `outputs/funding_channel_analysis/`.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.api as sm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
ROOT = Path('.')
basis_df = pd.read_csv(ROOT / '_data' / 'mispricing_basis.csv', parse_dates=['date']).rename(columns={'tenor': 'tenor_bucket'})
weekly_basis = basis_df.set_index('date').groupby('tenor_bucket')['basis'].resample('W-FRI').mean().reset_index()
weekly_basis.head()

### Rolling AR(1) computation

In [ ]:
WINDOW = 26
records = []
for tenor, grp in weekly_basis.groupby('tenor_bucket'):
    grp = grp.sort_values('date').copy()
    grp['basis_lag1'] = grp['basis'].shift(1)
    for idx in range(WINDOW, len(grp)):
        window = grp.iloc[idx-WINDOW+1:idx+1]
        if window['basis_lag1'].isna().any():
            continue
        X = sm.add_constant(window['basis_lag1'])
        model = sm.OLS(window['basis'], X).fit()
        phi = model.params['basis_lag1']
        hl = -np.log(2) / np.log(phi) if 0 < phi < 1 else np.nan
        records.append({
            'date': window['date'].iloc[-1],
            'tenor_bucket': tenor,
            'ar1_rolling': phi,
            'half_life_rolling': hl,
            'window_nobs': int(model.nobs)
        })
rolling_df = pd.DataFrame(records)
rolling_df.head()

### Funding proxies

In [ ]:
funding_df = pd.read_csv(ROOT / 'inputs' / 'funding_channels' / 'cip_basis_series.csv', parse_dates=['date'])
wedge_df = pd.read_csv(ROOT / 'inputs' / 'funding_channels' / 'money_market_wedge.csv', parse_dates=['date'])
swap_df = pd.read_csv(ROOT / 'inputs' / 'funding_channels' / 'swap_spread_proxy.csv', parse_dates=['date'])
funding_weekly = funding_df.merge(wedge_df, on='date', how='outer').merge(swap_df, on='date', how='outer').set_index('date').resample('W-FRI').mean().reset_index()
funding_weekly.head()

### Merge controls and compute Funding PC

In [ ]:
liq_df = pd.read_csv(ROOT / 'outputs' / 'strategy3' / 'concentration_metrics.csv', parse_dates=['qdate']).rename(columns={'qdate': 'date'})
liq_weekly = liq_df.set_index('date').groupby('tenor_bucket')[['liq_hhi','issue_conc_top3','issue_conc_top5','pubout','n_issues']].resample('W-FRI').mean().reset_index()
agg_weekly = liq_df.set_index('date')[['liq_hhi','pubout','n_issues','issue_conc_top3','issue_conc_top5']].resample('W-FRI').agg({'liq_hhi':'mean','pubout':'sum','n_issues':'sum','issue_conc_top3':'mean','issue_conc_top5':'mean'}).reset_index()
panel = rolling_df.merge(liq_weekly, on=['date','tenor_bucket'], how='left').merge(agg_weekly, on='date', how='left').merge(funding_weekly, on='date', how='left')
fund_cols = ['CIP_basis_USDJPY','CIP_basis_EURUSD','RRP_bill_wedge','swap_spread_10y']
valid = panel.dropna(subset=fund_cols).drop_duplicates(subset='date')
scaler = StandardScaler()
pca = PCA(n_components=1)
valid['Funding_PC'] = pca.fit_transform(scaler.fit_transform(valid[fund_cols]))
panel = panel.merge(valid[['date','Funding_PC']], on='date', how='left')
panel.head()

### Regression example

In [ ]:
controls = ['liq_hhi','agg_liq_hhi','agg_pubout','agg_n_issues']
reg_sample = panel.dropna(subset=['half_life_rolling'] + fund_cols + controls)
reg_sample['quarter_end'] = reg_sample['date'].dt.is_quarter_end.astype(int)
example = reg_sample[reg_sample['tenor_bucket']==10]
X = sm.add_constant(example[fund_cols + controls + ['quarter_end']])
model = sm.OLS(example['half_life_rolling'], X).fit()
print(model.summary().tables[1])

### Export artifacts

In [ ]:
panel.to_csv('outputs/funding_channel_analysis/merged_funding_panel_preview.csv', index=False)
panel.describe().to_csv('outputs/funding_channel_analysis/merged_funding_panel_descriptive.csv')
print('Preview and descriptive stats saved.')